# Study BFGS method
Pure implementation (`numpy` only) is based on the book "Numerical Optimization" by Jorge Nocedal and Stephen J. Wright.

In [1]:
import numpy as np

from numpy.linalg import norm

## Introduce line search function

In [2]:
def line_search(phi, alpha_max=None, maxiter=10):
    """
    Line search algorithm that finds suitable step length satisfying the strong Wolfe conditions.
    Implemented accirdingly to the book "Numerical Optimization" by Jorge Nocedal and Stephen J. Wright.
    """
    eps = 1e-6
    c1, c2 = 1e-4, 0.9 # Constant from the book
    alpha = [0, 1 if not alpha_max else min(1, alpha_max)]
    dphi = lambda alpha: (phi(alpha + eps) - phi(alpha - eps)) / (2 * eps)

    for k in range(1, maxiter):
        if (
            phi(alpha[k]) > phi(0) + c1 * alpha[k] * dphi(0)
            or (k > 1 and phi(alpha[k]) > phi(alpha[k - 1]))
        ):
            return zoom(phi, dphi, alpha[k - 1], alpha[k])
        
        if abs(dphi(alpha[k])) <= -c2 * dphi(0):
            return alpha[k]
        
        if dphi(alpha[k]) >= 0:
            return zoom(phi, dphi, alpha[k], alpha[k - 1])
        
        alpha.append(2 * alpha[k] if not alpha_max else min(2 * alpha[k], alpha_max))

    return alpha[-1]

def quadratic_interpolation_minimizer(a, fa, dfa, b, fb):
    return a - (dfa * ((b - a) ** 2)) / (2.0 * (fb - fa - dfa * (b - a)))

def zoom(phi, dphi, alpha_low, alpha_hight):
    c1, c2 = 1e-4, 0.9 # Constant from the book
    while True:
        alpha = quadratic_interpolation_minimizer(
            alpha_low, phi(alpha_low), dphi(alpha_low), alpha_hight, phi(alpha_hight)
        )
        
        # Check for bisection
        if abs(alpha - alpha_low) < 0.1 * (alpha_hight - alpha_low):
            alpha = 0.5 * (alpha_hight - alpha_low)
        
        if phi(alpha) > phi(0) + c1 * alpha * dphi(0) or phi(alpha) >= phi(alpha_low):
            alpha_hight = alpha
        else:
            if abs(dphi(alpha)) <= -c2 * dphi(0):
                return alpha
            if dphi(alpha) * (alpha_hight - alpha_low) >= 0:
                alpha_hight = alpha_low
            alpha_low = alpha

### Test line search

In [3]:
eps = 1e-6
phi = lambda alpha: (10*alpha - 1) ** 4 + np.sin(alpha)
dphi = lambda alpha: (phi(alpha + eps) - phi(alpha - eps)) / (2 * eps)

In [4]:
c1, c2 = 1e-4, 0.9 # Constant from the book
result = line_search(phi, alpha_max=None, maxiter=10)
# Check Wolfe conditions
print(phi(result) <= phi(0) + c1 * result * dphi(0), abs(dphi(result)) <= c2 * abs(dphi(0)))

True True


In [5]:
result

0.08668266892653413

## BFGS

In [6]:
def compute_gradient(fun, x, eps=1e-6):
    """
    Numerical approximation of the gradient with two points central difference.
    """
    
    x = np.array(x).astype('float64')
    gradient = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        x_plus, x_minus = x.copy(), x.copy()
        x_plus[i] += eps
        x_minus[i] -= eps
        gradient[i] = (fun(x_plus) - fun(x_minus)) / (2 * eps)
    
    return gradient

In [7]:
def bfgs(fun, x0, tol=None, gtol=None, grad=None, eps=None, maxiter=None):
    """
    Minimize function using BFGS method (an analog of `scipy.optimize.minimize(method='BFGS')`).
    
    Parameters
    ----------
    fun : callable
        Target function to be minimized.
    x0 : ndarray, (n,)
        Initial guess.
    tol : float, >0
        Target tolarance to stop iterations, $||x_{k} - x_{k - 1}|| < tol$.
    gtol : float, >0
        Target gradient tolarance to sotp iterations, $||||$
    gradient : collable
        Function to compute gradient, `grad(x) -> ndarray`.
    eps : float
        Step size for numerical approximation of the gradient (if `grad` is not specified).
    """
    
    if not grad:
        grad = lambda x: compute_gradient(fun, x)
    
    if not gtol:
        gtol = 1e-9
    
    k = 0
    x = [x0]
    
    x0 = np.array(x0)
    identity_matrix = np.identity(x0.shape[0])
    
    # Initial approximation of the inverse Hessian matrix
    inverse_hessian = identity_matrix.copy()
    
    while norm(grad(x[k])) > gtol:
        if maxiter and k == maxiter:
            break
        
        # Compute search direction
        p = -inverse_hessian.dot(grad(x[k]))
        
        # Compute next value `x[k + 1]` from a line search procedure
        # to satisfy the Wolfe conditions
        phi = lambda alpha: fun(x[k] + alpha * p)
        alpha = line_search(phi)
        
        # Compare with `scipy` implementation of `line_search`
        # print(optimize.line_search(fun, grad, x[k], p))
        # alpha = optimize.line_search(fun, grad, x[k], p)[0]
        
        x.append(x[k] + alpha * p)
        
        if tol and norm(x[k + 1] - x[k]) < tol:
            break
        
        # Update inverse Hessian matrix
        s = x[k + 1] - x[k]
        y = grad(x[k + 1]) - grad(x[k])
        
        rho = 1 / np.dot(y, s)
        
        inverse_hessian = (identity_matrix - rho * s[..., None] * y).dot(
            inverse_hessian.dot(
                identity_matrix - rho * y[..., None] * s)) + (rho * s[..., None] * s)
        
        # Next iteration
        k += 1
        
    return {'x': x[-1], 'hess_inv': inverse_hessian, 'nit': k}

### Test BFGS on Rosenbrock function

In [8]:
fun = lambda x: (1 - x[0]) ** 2 + 100 * (x[1] - x[0] ** 2) ** 2

In [9]:
result = bfgs(fun, [4, 2], gtol=1e-6)
result

{'x': array([1.        , 0.99999999]),
 'hess_inv': array([[0.49979725, 0.99964772],
        [0.99964772, 2.00438779]]),
 'nit': 25}

### SciPy implementation

In [10]:
from scipy.optimize import minimize

result = minimize(fun, [4, 2], jac=lambda x: compute_gradient(fun, x), method='BFGS', options={'gtol': 1e-6})
result

      fun: 5.7421750691157386e-18
 hess_inv: array([[0.50201727, 1.00397784],
       [1.00397784, 2.01284373]])
      jac: array([-2.91032138e-08,  1.24374178e-08])
  message: 'Optimization terminated successfully.'
     nfev: 65
      nit: 50
     njev: 65
   status: 0
  success: True
        x: array([1., 1.])